# Define API keys and libraries

In [113]:
import json
import requests

import pandas as pd

In [ ]:
api_key = 'da56733a-6d62-4da1-9d2f-4e882d46478b'

# Define functions for calling API endpoints

In [142]:
class api_511:
    
    def __init__(self, api_key):
        self.api_key = api_key
    
    def get_request(self, url, params):
        response = requests.get(
            url = url,
            params = {
                'api_key': self.api_key,
                **params
            }
        )
        decoded = response.content.decode('utf-8-sig') # strip byte order mark from response
        return json.loads(decoded)
    
    def get_stops(self, operator_id='SF'):
        response = self.get_request(
            url = 'http://api.511.org/transit/stops',
            params = {
                'format': 'json',
                'operator_id': operator_id
            }
        )
        stop_list = response['Contents']['dataObjects']['ScheduledStopPoint']
        stop_df = pd.DataFrame([
            {
                'stop_id': d['id'],
                'name': d['Name'],
                'longitude': d['Location']['Longitude'],
                'latitude': d['Location']['Latitude'],
                'url': d['Url'],
                'stop_type': d['StopType']
            } for d in stop_list
        ])
        
        return stop_df
    
    def get_stop_monitoring(self, route, stop_code, operator_id='SF'):
        response = self.get_request(
            url = 'http://api.511.org/transit/StopMonitoring',
            params = {
                'format': 'json',
                'agency': operator_id,
                'route': route,
                'stopCode': stop_code
            }
        )
        
        monitor_list = response['ServiceDelivery']['StopMonitoringDelivery']['MonitoredStopVisit']
        monitor_df = pd.DataFrame([
            {
                'timestamp': d['RecordedAtTime'],
                'line': d['MonitoredVehicleJourney']['LineRef'],
                'line_name': d['MonitoredVehicleJourney']['PublishedLineName'],
                'origin_stop_id': d['MonitoredVehicleJourney']['OriginRef'],
                'origin_stop_name': d['MonitoredVehicleJourney']['OriginName'],
                'destination_stop_id': d['MonitoredVehicleJourney']['DestinationRef'],
                'destination_stop_name': d['MonitoredVehicleJourney']['DestinationName'],
                'monitored_stop_id': d['MonitoredVehicleJourney']['MonitoredCall']['StopPointRef'],
                'monitored_stop_name': d['MonitoredVehicleJourney']['MonitoredCall']['StopPointName'],
                'is_monitored': d['MonitoredVehicleJourney']['Monitored'],
                'vehicle_longitude': d['MonitoredVehicleJourney']['VehicleLocation']['Longitude'],
                'vehicle_latitude': d['MonitoredVehicleJourney']['VehicleLocation']['Latitude'],
                'aimed_arrival_time': d['MonitoredVehicleJourney']['MonitoredCall']['AimedArrivalTime'],
                'expected_arrival_time': d['MonitoredVehicleJourney']['MonitoredCall']['ExpectedArrivalTime'],
            } for d in monitor_list
        ])
        
        return monitor_df

In [143]:
api = api_511(api_key)

In [119]:
# List the stops available
stops = api.get_stops()

In [120]:
stops

,stop_id,name,longitude,latitude,url,stop_type
0,15829,100 O'Shaughnessy Blvd,-122.450678,37.744481,https://www.sfmta.com/15829,onstreetBus
1,13230,10th Ave & Ortega St,-122.466502,37.752724,https://www.sfmta.com/13230,onstreetBus
2,13231,10th Ave & Pacheco St,-122.466376,37.75086,https://www.sfmta.com/13231,onstreetBus
3,13232,10th Ave & Pacheco St,-122.466547,37.750699,https://www.sfmta.com/13232,onstreetBus
4,13234,10th Ave & Quintara St,-122.466432,37.749164,https://www.sfmta.com/13234,onstreetBus
...,...,...,...,...,...,...
3276,16988,Yerba Buena Ave & Hazelwood Ave,-122.459705,37.734389,https://www.sfmta.com/16988,onstreetBus
3277,16989,Yerba Buena Ave & Ravenwood Dr,-122.46055,37.73531,https://www.sfmta.com/16989,onstreetBus
3278,16990,Yerba Buena Ave & Saint Elmo Way,-122.458719,37.732623,https://www.sfmta.com/16990,onstreetBus
3279,17878,Yerba Buena/Moscone Station Northbound,-122.401551,37.782358,https://www.sfmta.com/17878,onstreetBus


In [144]:
# Get the live predictions for a stop
route = '48'
stop_code = '16121' # Portola Dr & Teresita Blvd
resp = api.get_stop_monitoring(route, stop_code)

In [145]:
resp

,timestamp,line,line_name,origin_stop_id,origin_stop_name,destination_stop_id,destination_stop_name,monitored_stop_id,monitored_stop_name,is_monitored,vehicle_longitude,vehicle_latitude,aimed_arrival_time,expected_arrival_time
0,2023-01-18T02:18:30Z,48,QUINTARA-24TH STREET,14781,Lower Great Hwy & Rivera St,13411,3rd St + 20th St,16121,Portola Dr & Teresita Blvd,True,-122.465538,37.7407341,2023-01-18T02:21:25Z,2023-01-18T02:22:20Z
1,1970-01-01T00:00:00Z,48,QUINTARA-24TH STREET,14781,Lower Great Hwy & Rivera St,13411,3rd St + 20th St,16121,Portola Dr & Teresita Blvd,True,,,2023-01-18T02:41:25Z,2023-01-18T02:41:11Z
2,1970-01-01T00:00:00Z,48,QUINTARA-24TH STREET,14781,Lower Great Hwy & Rivera St,13411,3rd St + 20th St,16121,Portola Dr & Teresita Blvd,True,,,2023-01-18T03:01:25Z,2023-01-18T03:01:11Z
